# Proyecto (obtención dataset)

In [10]:
# --- General ---
import os

# --- Data ---
import pandas as pd
# from pandasql import sqldf
# import numpy as np
# import statistics as stats

# --- Conexión ---
import elasticsearch

# --- Procesamiento lenguaje: spacy ---
import spacy
# from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

# --- Procesamiento lenguaje: gensim ---
# import gensim
# import gensim.corpora as corpora
# from gensim.models import CoherenceModel

# --- Prediccion Positividad: POS, NEU, NEG ---
# from tqdm import tqdm
# import transformers
# from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# --- Visualización ---
# import pyLDAvis
# import pyLDAvis.gensim_models 
# import matplotlib.colors as colors
# import matplotlib.pyplot as plt
# import geopandas as gpd
# %matplotlib inline

# ==============================================================================

# --- Configuración ---
# User password
password = os.environ.get('SOPHIA2')

# - - - geoData - - -
import tools_region as tr

# --- Funciones ---
# Cargar paquete de español mediano
nlp = spacy.load("es_core_news_md")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Utilización de regiones y comunas

In [11]:
data_regiones_comunas = tr.search_region('Región_de_Los_Lagos')
if(data_regiones_comunas):
    region = data_regiones_comunas['region']
    comunas_sql = data_regiones_comunas['comunas']
    comunas = [comuna.replace('_', ' ') for comuna in comunas_sql]

Region encontrada


## Inicialización peticiones Sophia2

In [12]:
# --- [Parametros de búsqueda] ---
# Búsqueda de palabra clave
keyword="empleo empleos trabajo trabajos empleabilidad contratación"
simple_keyword= True

# -- Nombre archivo --
nombre_archivo = 'data_Region_X_EMPLEO_' # Resultado: ./data/{nombre_archivo}_EMPLEO_{from_}_{to_}.csv

# -- Parameters queries --
# País
country="chile"
# Fecha
from_="2022-01-01"
to_="2022-06-30"
# Medios de prensa - Region X
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

print(nombre_archivo)

data_Region_X_EMPLEO_


In [13]:
# --- Data Connection ---
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS=  password

# --- Connection ---
es = elasticsearch.Elasticsearch(
    IP,
    http_auth=(USER, PASS)
)

# --- keyword is phrase or just words---

match=""
if (simple_keyword):
    match="match"
else:
    match="match_phrase"

# --- Queries ---
query = { 
    "bool": { 
     "must": [
        {match: { "text":keyword}}

      ],
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }
      }},

        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

In [14]:
# --- Confirmación de conexión ---
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 1652 noticias encontradas...


## Creación dataframe

In [15]:
# --- Columnas Dataframe ---
data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}
df = pd.DataFrame(data)  

# --- Información Dataframe---
for hit in res['hits']['hits']:

    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':int(id_news), 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
    df = pd.concat([df, pd.DataFrame.from_records([new_row])])
    # df = df.append(new_row, ignore_index=True)


# Reseteamos el índice del dataframe para que sea secuencial, sino serían puros 0.
df = df.reset_index(drop=True)

# --- Elimina resiuduos ---
# Elimina duplicados
df = df.drop_duplicates(subset='url', keep='first')
# Elimina textos vacíos o con muy poco contenido
df = df[df['text'].str.len() > 50]

In [16]:
print(f"Son {len(df)} noticias encontradas que se utilizarán")
df.head()

Son 1642 noticias encontradas que se utilizarán


,id_news,country,media_outlet,url,title,text,date
0,47316734.0,chile,seminariolocal,https://www.semanariolocal.cl/archivos/6420,Parlamentario pide incentivar contratación de ...,Un aumento de mercados laborales poco dispuest...,2022-06-06
1,22598469.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2022/05/03/...,En medio de inestabilidad económica preocupa p...,"Casi 415 mil personas estaban empleadas, en pr...",2022-05-03
2,47315857.0,chile,seminariolocal,https://www.semanariolocal.cl/archivos/6842,Solicitan extender IFE Laboral en mujeres para...,Luego que el Instituto Nacional de Estadística...,2022-06-29
3,47587124.0,chile,radiosago,https://www.radiosago.cl/tasa-de-desocupacion-...,Tasa de desocupación en la región de Los Lagos...,"Según la Encuesta Nacional de Empleo (ENE), qu...",2022-05-30
4,22598388.0,chile,radioacogida,https://radioacogida.cl/sence-afianza-trabajo-...,SENCE AFIANZA TRABAJO CON OFICINAS OMIL DE LA ...,"En San Pablo, provincia de Osorno y en Chonchi...",2022-05-11


## Guardamos las noticias sin menciones a comunas

In [17]:
# --- Guarda archivo ---
# print(nombre_archivo+from_+"_"+to_+'.csv')
# df.to_csv("./data/"+nombre_archivo+from_+"_"+to_+".csv")

## Añadiremos las comunas mencionadas en las noticias

In [18]:
# --- Creamos Matcher utilizando nlp ( natural language processing ) ---
matcher_comunas = PhraseMatcher(nlp.vocab)
for comuna in comunas:
    matcher_comunas.add(comuna, [nlp(comuna)])

# --- Función para buscar comunas ---
for index,row in df.iterrows():
    txt = row["text"]
    
    try:
        # -- Procesamos el texto --
        doc = nlp(txt)
        matches_comunas = matcher_comunas(doc)

        # -- Creamos lista de comunas --
        for match_id, start, end in matches_comunas:
            span = doc[start:end]  # The matched span

    except:
        pass

In [19]:
# --- Reemplazamos los espacios por guiones bajos, para cada Comuna ---
# ejemplo: "San Juan de la Costa" -> "San_Juan_de_la_Costa"
df_comunas = df
for comuna in comunas:
    df_comunas.insert(7,comuna.replace(" ","_"),0)

# --- Insertamos las comunas al dataframe (df) ---
for index,row in df_comunas.iterrows():
    txt = row["text"]
    
    try:
    
        doc = nlp(txt)
        matches_comunas = matcher_comunas(doc)

        for match_id, start, end in matches_comunas:
            span = doc[start:end]  # The matched span
            df_comunas.at[index,span.text.replace(" ","_")]=1
    except Exception as e:
        print(e)
        pass

## Guardamos la información CSV

In [20]:
# --- Guarda archivo ---
print(nombre_archivo+'comunas_'+from_+"_"+to_+".csv")
df_comunas.to_csv("./data/"+nombre_archivo+'comunas_'+from_+"_"+to_+".csv")

data_Region_X_EMPLEO_comunas_2022-01-01_2022-06-30.csv


###  Con esto tenemos toda la información necesaria para cargar los datos y utilizarlos.